<a href="https://colab.research.google.com/github/Leerish/Captcha-Decoding-using-TensorFlow/blob/main/Captcha_Decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Captcha to Text OCR using TensorFlow


**Introduction:**

**Captcha (Completely Automated Public Turing test to tell Computers and Humans Apart)** is a challenge-response test used in computing to determine whether or not the user is human. It typically involves presenting the user with a distorted image of text, and the user must correctly transcribe the text to prove they are human. Captchas are commonly used in online forms, website sign-ups, and account logins to prevent automated bots from abusing or spamming the system.


The model is designed to read and interpret captcha images using TensorFlow. It utilizes deep learning techniques, particularly convolutional neural networks (CNNs), to recognize the distorted text within captcha images. By training on a dataset of labeled captcha images, the model learns to extract features and classify the characters present in the images. Once trained, the model can then be used to predict the text present in unseen captcha images.

Key components of the captcha reading model may include:

1. **Data Preprocessing**: Captcha images are preprocessed to enhance their quality and make them suitable for training. Preprocessing steps may include resizing, normalization, noise reduction, and augmentation.

2. **Model Architecture**: The model architecture defines the structure of the neural network used for captcha recognition. It typically consists of multiple layers, including convolutional layers for feature extraction, pooling layers for downsampling, and fully connected layers for classification.

3. **Training**: The model is trained on a dataset of labeled captcha images. During training, the model learns to minimize the difference between its predictions and the ground truth labels using optimization algorithms like stochastic gradient descent (SGD) or Adam.

4. **Evaluation**: The trained model is evaluated on a separate validation dataset to assess its performance. Evaluation metrics such as accuracy, precision, recall, and F1 score are used to measure the model's effectiveness in captcha recognition.

5. **Deployment**: Once trained and evaluated, the model can be deployed in production environments to read captcha images in real-time. This may involve integrating the model into web applications, APIs, or other systems where captcha verification is required.

Overall, the captcha reading model using TensorFlow demonstrates the application of deep learning techniques in solving practical challenges related to computer vision and security. It provides an automated solution for reading and interpreting captcha images, contributing to improved user experience and security in online applications.

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-04-17 14:50:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-04-17 14:50:40 (20.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



##Setting up initial Configurations

In [4]:
import os
from datetime import datetime

from mltu.configs import BaseModelConfigs


class ModelConfigs(BaseModelConfigs):
    def __init__(self):
        super().__init__()
        self.model_path = os.path.join("Models/02_captcha_to_text", datetime.strftime(datetime.now(), "%Y%m%d%H%M"))
        self.vocab = ""
        self.height = 50
        self.width = 200
        self.max_text_length = 0
        self.batch_size = 64
        self.learning_rate = 1e-3
        self.train_epochs = 70
        self.train_workers = 20

## Installing Dependencies

In [4]:
! pip install stow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 1.4 MB/s eta 0:00:00


In [1]:
!pip install mltu==0.1.4

## Data Preprocesing

In [2]:
import stow
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

def download_and_unzip(url, extract_to='Datasets'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

if not stow.exists(stow.join('Datasets', 'captcha_images_v2')):
    download_and_unzip('https://github.com/AakashKumarNain/CaptchaCracker/raw/master/captcha_images_v2.zip', extract_to='Datasets')

In [6]:
dataset, vocab, max_len = [], set(), 0
captcha_path = os.path.join("Datasets", "captcha_images_v2")
for file in os.listdir(captcha_path):
    file_path = os.path.join(captcha_path, file)
    label = os.path.splitext(file)[0] # Get the file name without the extension
    dataset.append([file_path, label])
    vocab.update(list(label))
    max_len = max(max_len, len(label))

configs = ModelConfigs()

# Save vocab and maximum text length to configs
configs.vocab = "".join(vocab)
configs.max_text_length = max_len
configs.save()

In [17]:
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from mltu.dataProvider import DataProvider
from mltu.losses import CTCloss
from mltu.callbacks import Model2onnx, TrainLogger
from mltu.metrics import CWERMetric

from mltu.preprocessors import ImageReader
from mltu.transformers import ImageResizer, LabelIndexer, LabelPadding
from mltu.augmentors import RandomBrightness, RandomRotate, RandomErodeDilate

from model import train_model
from configs import ModelConfigs

import os
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


In [9]:
from mltu.dataProvider import DataProvider
from mltu.preprocessors import ImageReader
from mltu.transformers import ImageResizer, LabelIndexer, LabelPadding
from mltu.augmentors import RandomBrightness, RandomRotate, RandomErodeDilate

data_provider = DataProvider(
    dataset=dataset,
    skip_validation=True,
    batch_size=configs.batch_size,
    data_preprocessors=[ImageReader()],
    transformers=[
        ImageResizer(configs.width, configs.height),
        LabelIndexer(configs.vocab),
        LabelPadding(max_word_length=configs.max_text_length, padding_value=len(configs.vocab))
        ],
)

INFO:mltu.dataProvider:Skipping Dataset validation...


In [10]:
train_data_provider, val_data_provider = data_provider.split(split = 0.9)

In [11]:
train_data_provider.augmentors = [RandomBrightness(), RandomRotate(), RandomErodeDilate()]

## Training Model


In [12]:
model = train_model(
    input_dim = (configs.height, configs.width, 3),
    output_dim = len(configs.vocab),
)

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=configs.learning_rate),
    loss=CTCloss(),
    metrics=[CWERMetric()],
)

In [14]:
# Define callbacks
earlystopper = EarlyStopping(monitor='val_CER', patience=40, verbose=1)
checkpoint = ModelCheckpoint(f"{configs.model_path}/model.h5", monitor='val_CER', verbose=1, save_best_only=True, mode='min')
trainLogger = TrainLogger(configs.model_path)
tb_callback = TensorBoard(f'{configs.model_path}/logs', update_freq=1)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_CER', factor=0.9, min_delta=1e-10, patience=20, verbose=1, mode='auto')
model2onnx = Model2onnx(f"{configs.model_path}/model.h5")

In [19]:
# Train the model
model.fit(
    train_data_provider,
    validation_data=val_data_provider,
    epochs=70,
    callbacks=[earlystopper, checkpoint, trainLogger, reduceLROnPlat, tb_callback, model2onnx],
    workers=configs.train_workers
)

# Save training and validation datasets as csv files
train_data_provider.to_csv(stow.join(configs.model_path, 'train.csv'))
val_data_provider.to_csv(stow.join(configs.model_path, 'val.csv'))

Epoch 1/70
15/15 [==============================] - ETA: 0s - loss: 16.3212 - CER: 1.0000 - WER: 1.0000
Epoch 1: val_CER did not improve from 1.00000


INFO:root:Epoch 0; loss: 16.321199417114258; CER: 1.0; WER: 1.0; val_loss: 16.35167121887207; val_CER: 1.0; val_WER: 1.0


15/15 [==============================] - 48s 3s/step - loss: 16.3212 - CER: 1.0000 - WER: 1.0000 - val_loss: 16.3517 - val_CER: 1.0000 - val_WER: 1.0000 - lr: 0.0010
Epoch 2/70
15/15 [==============================] - ETA: 0s - loss: 16.3149 - CER: 1.0000 - WER: 1.0000
Epoch 2: val_CER did not improve from 1.00000


INFO:root:Epoch 1; loss: 16.31488609313965; CER: 1.0; WER: 1.0; val_loss: 16.328990936279297; val_CER: 1.0; val_WER: 1.0


15/15 [==============================] - 48s 3s/step - loss: 16.3149 - CER: 1.0000 - WER: 1.0000 - val_loss: 16.3290 - val_CER: 1.0000 - val_WER: 1.0000 - lr: 0.0010
Epoch 3/70
15/15 [==============================] - ETA: 0s - loss: 16.3159 - CER: 1.0000 - WER: 1.0000
Epoch 3: val_CER did not improve from 1.00000


...

INFO:root:Epoch 68; loss: 0.46646228432655334; CER: 0.027350427582859993; WER: 0.13247863948345184; val_loss: 1.0080604553222656; val_CER: 0.04999999701976776; val_WER: 0.24038460850715637


15/15 [==============================] - 46s 3s/step - loss: 0.4665 - CER: 0.0287 - WER: 0.1395 - val_loss: 1.0081 - val_CER: 0.0500 - val_WER: 0.2404 - lr: 9.0000e-04
Epoch 70/70
15/15 [==============================] - ETA: 0s - loss: 0.5082 - CER: 0.0289 - WER: 0.1399
Epoch 70: val_CER did not improve from 0.04423


INFO:root:Epoch 69; loss: 0.5081627368927002; CER: 0.028418803587555885; WER: 0.1378205120563507; val_loss: 1.5910942554473877; val_CER: 0.07499999552965164; val_WER: 0.3461538553237915


15/15 [==============================] - 46s 3s/step - loss: 0.5082 - CER: 0.0288 - WER: 0.1398 - val_loss: 1.5911 - val_CER: 0.0750 - val_WER: 0.3462 - lr: 9.0000e-04


INFO:tf2onnx.tfonnx:Using tensorflow=2.10.1, onnx=1.12.0, tf2onnx=1.14.0/8f8d49
INFO:tf2onnx.tfonnx:Using opset <onnx, 15>
INFO:tf2onnx.tf_utils:Computed 0 values for constant folding
INFO:tf2onnx.tf_utils:Computed 0 values for constant folding
INFO:tf2onnx.tf_utils:Computed 0 values for constant folding
INFO:tf2onnx.tf_utils:Computed 0 values for constant folding
INFO:tf2onnx.tf_utils:Computed 2 values for constant folding
INFO:tf2onnx.tfonnx:folding node using tf type=StridedSlice, name=model/bidirectional/forward_lstm/PartitionedCall/strided_slice
INFO:tf2onnx.tfonnx:folding node using tf type=StridedSlice, name=model/bidirectional/backward_lstm/PartitionedCall/strided_slice
INFO:tf2onnx.optimizer:Optimizing ONNX model
INFO:tf2onnx.optimizer:After optimization: BatchNormalization -18 (18->0), Cast -4 (11->7), Concat -4 (10->6), Const -128 (194->66), Expand -3 (4->1), Gather +2 (2->4), Identity -2 (2->0), Shape -1 (4->3), Slice -1 (5->4), Squeeze -3 (5->2), Transpose -83 (87->4), Uns

In [22]:
! pip install text-utils

In [5]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.5 MB/s eta 0:00:00


# Verifying Results

In [5]:
import cv2
import typing
import numpy as np
import Levenshtein

from mltu.inferenceModel import OnnxInferenceModel

class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, char_list: typing.Union[str, list], *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.char_list = char_list

    @staticmethod
    def ctc_decoder(preds, char_list):
        decoded_texts = []

        for pred in preds:
            decoded_text = ''
            previous_index = len(char_list)  # Initialize previous_index to an invalid value

            for timestep_pred in pred:
                max_index = np.argmax(timestep_pred)

                # Check if the max_index is valid and not the blank index
                if 0 <= max_index < len(char_list) and max_index != len(char_list) - 1:
                    # Append the character to the decoded text
                    decoded_text += char_list[max_index]

                # Update previous_index
                previous_index = max_index

            decoded_texts.append(decoded_text)

        return decoded_texts

    @staticmethod
    def get_cer(prediction_text, label):
       # Ensure both prediction_text and label are strings
        if not isinstance(prediction_text, str) or not isinstance(label, str):
            raise ValueError("Both prediction_text and label must be strings")

        # Initialize variables to store counts of insertions, deletions, and substitutions
        insertions = deletions = substitutions = 0

        # Lengths of prediction_text and label
        len_pred = len(prediction_text)
        len_label = len(label)

        # Initialize matrices for dynamic programming approach
        dp = [[0] * (len_label + 1) for _ in range(len_pred + 1)]

        # Initialize first row and column of dp matrix
        for i in range(len_pred + 1):
            dp[i][0] = i
        for j in range(len_label + 1):
            dp[0][j] = j

        # Calculate edit distance using dynamic programming
        for i in range(1, len_pred + 1):
            for j in range(1, len_label + 1):
                if prediction_text[i - 1] == label[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

        # Edit distance is the last cell in the matrix
        edit_distance = dp[len_pred][len_label]

        # Calculate CER
        cer = edit_distance / len_label

        return cer

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shape[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = self.ctc_decoder(preds, self.char_list)[0]

        return text

if __name__ == "__main__":
    import pandas as pd
    from tqdm import tqdm
    from mltu.configs import BaseModelConfigs

    configs = BaseModelConfigs.load("/content/Models/02_captcha_to_text/202404171552/configs.yaml")

    model = ImageToWordModel(model_path=configs.model_path, char_list=configs.vocab)

    df = pd.read_csv("/content/Models/02_captcha_to_text/202404171552/val.csv").values.tolist()

    accum_cer = []
    for image_path, label in tqdm(df):
        image = cv2.imread(image_path)

        prediction_text = model.predict(image)

        cer = model.get_cer(prediction_text, label)
        print(f"Image: {image_path}, Label: {label}, Prediction: {prediction_text}, CER: {cer}")

        accum_cer.append(cer)

    print(f"Average CER: {np.average(accum_cer)}")


 15%|█▌        | 16/104 [00:00<00:00, 154.37it/s]

Image: Datasets/captcha_images_v2/3ebnn.png, Label: 3ebnn, Prediction: 3ebnn, CER: 0.0
Image: Datasets/captcha_images_v2/wf684.png, Label: wf684, Prediction: wf6844, CER: 0.2
Image: Datasets/captcha_images_v2/6pfy4.png, Label: 6pfy4, Prediction: 6pfy4, CER: 0.0
Image: Datasets/captcha_images_v2/n7ff2.png, Label: n7ff2, Prediction: n7ff2, CER: 0.0
Image: Datasets/captcha_images_v2/478nx.png, Label: 478nx, Prediction: 478nx, CER: 0.0
Image: Datasets/captcha_images_v2/nnfx3.png, Label: nnfx3, Prediction: nnfxx3, CER: 0.2
Image: Datasets/captcha_images_v2/cnmnn.png, Label: cnmnn, Prediction: nmnnn, CER: 0.4
Image: Datasets/captcha_images_v2/ddcne.png, Label: ddcne, Prediction: ddne, CER: 0.2
Image: Datasets/captcha_images_v2/6cm6m.png, Label: 6cm6m, Prediction: 6m6m, CER: 0.2
Image: Datasets/captcha_images_v2/b28g8.png, Label: b28g8, Prediction: b28g8, CER: 0.0
Image: Datasets/captcha_images_v2/5n245.png, Label: 5n245, Prediction: 5n245, CER: 0.0
Image: Datasets/captcha_images_v2/244e2.png

 45%|████▌     | 47/104 [00:00<00:00, 145.64it/s]

Image: Datasets/captcha_images_v2/2p2y8.png, Label: 2p2y8, Prediction: 2p2yy8, CER: 0.2
Image: Datasets/captcha_images_v2/gpnxn.png, Label: gpnxn, Prediction: gpnxn, CER: 0.0
Image: Datasets/captcha_images_v2/g8gnd.png, Label: g8gnd, Prediction: g8gnnd, CER: 0.2
Image: Datasets/captcha_images_v2/mbf58.png, Label: mbf58, Prediction: mbf588, CER: 0.2
Image: Datasets/captcha_images_v2/4ycex.png, Label: 4ycex, Prediction: 4yex, CER: 0.2
Image: Datasets/captcha_images_v2/2356g.png, Label: 2356g, Prediction: 2356g, CER: 0.0
Image: Datasets/captcha_images_v2/mg5nn.png, Label: mg5nn, Prediction: mg5nn, CER: 0.0
Image: Datasets/captcha_images_v2/4w6mw.png, Label: 4w6mw, Prediction: 4w6mw, CER: 0.0
Image: Datasets/captcha_images_v2/bgd4m.png, Label: bgd4m, Prediction: bgd4m, CER: 0.0
Image: Datasets/captcha_images_v2/44fyb.png, Label: 44fyb, Prediction: 44fyb, CER: 0.0
Image: Datasets/captcha_images_v2/24pew.png, Label: 24pew, Prediction: 74pew, CER: 0.2
Image: Datasets/captcha_images_v2/5bb66.p

 74%|███████▍  | 77/104 [00:00<00:00, 145.84it/s]

Image: Datasets/captcha_images_v2/yyn57.png, Label: yyn57, Prediction: yyn57, CER: 0.0
Image: Datasets/captcha_images_v2/77wp4.png, Label: 77wp4, Prediction: 77wp44, CER: 0.2
Image: Datasets/captcha_images_v2/n4xx5.png, Label: n4xx5, Prediction: n4xxx5, CER: 0.2
Image: Datasets/captcha_images_v2/b4y5x.png, Label: b4y5x, Prediction: b4y5xx, CER: 0.2
Image: Datasets/captcha_images_v2/eng53.png, Label: eng53, Prediction: eng53, CER: 0.0
Image: Datasets/captcha_images_v2/245y5.png, Label: 245y5, Prediction: 245yy5, CER: 0.2
Image: Datasets/captcha_images_v2/ef4mn.png, Label: ef4mn, Prediction: ef4nmn, CER: 0.2
Image: Datasets/captcha_images_v2/d22bd.png, Label: d22bd, Prediction: d22bd, CER: 0.0
Image: Datasets/captcha_images_v2/xe6eb.png, Label: xe6eb, Prediction: xe6eb, CER: 0.0
Image: Datasets/captcha_images_v2/mc35n.png, Label: mc35n, Prediction: m35n, CER: 0.2
Image: Datasets/captcha_images_v2/p2dw7.png, Label: p2dw7, Prediction: p2dw7, CER: 0.0
Image: Datasets/captcha_images_v2/8w875

100%|██████████| 104/104 [00:00<00:00, 147.66it/s]

Image: Datasets/captcha_images_v2/c7gb3.png, Label: c7gb3, Prediction: 7gb3, CER: 0.2
Image: Datasets/captcha_images_v2/e3ndn.png, Label: e3ndn, Prediction: e3ndn, CER: 0.0
Image: Datasets/captcha_images_v2/4w76g.png, Label: 4w76g, Prediction: 4w76gg, CER: 0.2
Image: Datasets/captcha_images_v2/m23bp.png, Label: m23bp, Prediction: m23bp, CER: 0.0
Image: Datasets/captcha_images_v2/pybee.png, Label: pybee, Prediction: pybeee, CER: 0.2
Image: Datasets/captcha_images_v2/d2nbn.png, Label: d2nbn, Prediction: d2nbn, CER: 0.0
Image: Datasets/captcha_images_v2/gbxyy.png, Label: gbxyy, Prediction: gbxyyy, CER: 0.2
Image: Datasets/captcha_images_v2/wb3ed.png, Label: wb3ed, Prediction: wb3ed, CER: 0.0
Image: Datasets/captcha_images_v2/42dw4.png, Label: 42dw4, Prediction: 42dw4, CER: 0.0
Image: Datasets/captcha_images_v2/5p8fm.png, Label: 5p8fm, Prediction: 3p8fm, CER: 0.2
Image: Datasets/captcha_images_v2/mfb3x.png, Label: mfb3x, Prediction: mfb3x, CER: 0.0
Image: Datasets/captcha_images_v2/87nym.p